In [1]:
import config
import pandas as pd
import psycopg2
import tweepy
import time

In [2]:
df = pd.read_csv('withSentiment_modified.csv', index_col=False)
df

,Tweet Id,Tweet Id Str,Tweet URL,Tweet Posted Time (UTC),Tweet Content,Tweet Type,Client,Retweets Received,Likes Received,Tweet Location,...,Name,Screen Name,User Bio,Verified or Non-Verified,Profile URL,Protected or Non-protected,User Followers,User Following,User Account Creation Date,sentiment
0,1233417783175778304,"""1233417783175778304""",https://twitter.com/Giussi92/status/1233417783...,2/28/2020 15:44,Also the entire Swiss Football League is on ho...,Tweet,Twitter for iPhone,0,0,NaN,...,Giuseppe Gentile,Giussi92,NaN,Verified,https://twitter.com/Giussi92,Non-Protected,3071,100,6/30/2013 0:27,negative
1,1233417742520332290,"""1233417742520332290""",https://twitter.com/LAMofficial/status/1233417...,2/28/2020 15:44,World Health Org Official: Trump’s press confe...,Tweet,Twitter Web App,0,0,Los Angeles CA USA,...,London After Midnight,LAMofficial,London After Midnight is a music project by wr...,Verified,https://twitter.com/LAMofficial,Non-Protected,4189,84,2/12/2010 21:43,positive
2,1233417741027225602,"""1233417741027225602""",https://twitter.com/mitchellvii/status/1233417...,2/28/2020 15:44,"I mean, Liberals are cheer-leading this #Coron...",Tweet,Twitter Web App,23,64,"Miami, FL",...,Bill Mitchell,mitchellvii,Host of YourVoice™ America at http://yourvoice...,Verified,https://twitter.com/mitchellvii,Non-Protected,534045,10037,12/9/2008 1:54,negative
3,1233417699264356357,"""1233417699264356357""",https://twitter.com/HelenKennedy/status/123341...,2/28/2020 15:44,"Under repeated questioning, Pompeo refuses to ...",Tweet,Twitter for iPhone,4,11,NYC and the North Fork,...,Helen Kennedy,HelenKennedy,Newspaperman (ex NY Daily News &amp; Boston He...,Verified,https://twitter.com/HelenKennedy,Non-Protected,27704,1060,3/25/2007 19:14,neutral
4,1233417674274807808,"""1233417674274807808""",https://twitter.com/W7VOA/status/1233417674274...,2/28/2020 15:44,#coronavirus comments now from @larry_kudlow h...,Tweet,Twitter for iPhone,0,0,James S. Brady Press Briefing Room,...,Steve Herman,W7VOA,@WhiteHouse bureau chief @VOANews. Member: @aa...,Verified,https://twitter.com/W7VOA,Non-Protected,94269,1999,12/6/2008 8:56,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60155,1204309839004012544,"""1204309839004012544""",https://twitter.com/RevistaArgos/status/120430...,12/10/2019 8:00,El #coronavirus entérico felino es un virus in...,Tweet,Hootsuite Inc.,0,0,NaN,...,Revista Argos,RevistaArgos,La revista de los veterinarios de animales de ...,Non-Verified,https://twitter.com/RevistaArgos,Non-Protected,4236,313,5/23/2012 14:30,neutral
60156,1204212528810729479,"""1204212528810729479""",https://twitter.com/RobNotRob11/status/1204212...,12/10/2019 1:33,"RT @timhquotes: It's my party, you're invited!...",ReTweet,Twitter for Android,0,0,NaN,...,Serving Suggestion,RobNotRob11,"6-and-a-half-foot tall, awkwardly shaped man. ...",Non-Verified,https://twitter.com/RobNotRob11,Non-Protected,37,259,1/6/2016 1:12,neutral
60157,1204206609708331009,"""1204206609708331009""",https://twitter.com/timhquotes/status/12042066...,12/10/2019 1:10,"It's my party, you're invited!\n\nPS, this is ...",Tweet,Twitter for Android,2,28,"Jackson Hole, WY",...,Tim Heidecker Quotes,timhquotes,Quotes from our favorite weird boy. Awesome pa...,Non-Verified,https://twitter.com/timhquotes,Non-Protected,2583,822,11/15/2013 3:20,neutral
60158,1201998948950577152,"""1201998948950577152""",https://twitter.com/Baric_Lab/status/120199894...,12/3/2019 22:57,Amy’s a survivor! #bariclab #pnnl #movingon #c...,Tweet,Twitter for iPhone,1,17,"Chapel Hill, NC",...,Baric Laboratory,Baric_Lab,"Researching SARS-CoV, MERS CoV, Dengue, Zika a...",Non-Verified,https://twitter.com/Baric_Lab,Non-Protected,1326,82,4/26/2018 15:07,neutral


In [3]:
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret



try:
    # create OAuthHandler object
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    # set access token and secret
    auth.set_access_token(access_token, access_token_secret)
    # create tweepy API object to fetch tweets
    api = tweepy.API(auth)
    print("Authentication Successful")
except:
    print("Error: Authentication Failed")
    
    

Authentication Successful


In [4]:
# Extract hashtags
def read_hashtags(tag_list):
    hashtags = []
    for tag in tag_list:
        hashtags.append(tag['text'])
    return hashtags

In [5]:
#def dbConnect(user_id, user_id_str, user_name, Screen_Name, location, verified, url, followers, following, user_creation_date,
#              tweet_id, tweet_url, created_at, tweet, retweet_count, likes, coord_lat,coord_long, language, sentiment
#             ):#hashtags):

def dbConnect(user_id,user_id_str,user_name,screen_name, location, verified,url, followers,following, user_creation_date, tweet_id, tweet_url, created_at, tweet):
             #tweet_id)#, tweet_url, created_at, tweet, retweet_count, likes):
    
    # Connection to database server
    
    #PGHOST = 'covidsentiment.cqciwtn1qpki.us-east-2.rds.amazonaws.com'
    
    conn = psycopg2.connect(host=config.PGHOST,database="postgres",port=5432,user='postgres',password=config.pgpassword)
    #conn = psycopg2.connect(host="localhost",database="postgres",port=5432,user='postgres',password=config.db_password)
    #conn = psycopg2.connect(host="localhost",database="TwitterDB",port=5432,user='postgres',password=config.db_password)
    
    cur = conn.cursor()

    # insert user information
    #command2 = '''INSERT INTO TwitterUser (user_id, user_id_str, user_name, screen_name, location, verified, url, followers, following, user_creation_date) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT
    #             (User_Id) DO NOTHING;'''
    command = '''INSERT INTO TwitterUser (user_id, user_id_str,user_name,screen_name, location, verified,url, followers,following,user_creation_date) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (User_ID) DO NOTHING'''
    
    
    #cur.execute(command,(user_id_str, user_name, Screen_Name, location, verified, url, followers, following, user_creation_date))
    cur.execute(command,(user_id,user_id_str,user_name,screen_name, location, verified,url, followers,following,user_creation_date))
    
    # insert tweet information
    command = '''INSERT INTO TwitterTweet (tweet_id,tweet_url, created_at, tweet, retweet_count, likes, coord_lat,coord_long, language, sentiment,user_id) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
    # retweet_count, likes, coord_lat,coord_long, language, sentiment) VALUES (%s,%s,%s,%s,%s,%s);'''
    cur.execute(command,(tweet_id,tweet_url, created_at, tweet,retweet_count, likes, coord_lat,coord_long, language, sentiment,user_id))#, created_at, tweet, retweet_count, likes))    
    
    # insert entity information
    for i in range(len(hashtags)):
        hashtag = hashtags[i]
        command = '''INSERT INTO TwitterEntity (tweet_id, hashtag) VALUES (%s,%s);'''
        cur.execute(command,(tweet_id, hashtag))
    
    # Commit changes
    conn.commit()
    
    # Disconnect
    cur.close()
    conn.close()
    
    

In [ ]:
for index,row in df.iterrows():
    tweet_id = int(df["Tweet Id"][index])
    tweet_id_str = df["Tweet Id Str"][index]
    tweet_url = df["Tweet URL"][index]
    created_at = df["Tweet Posted Time (UTC)"][index]
    tweet = df["Tweet Content"][index] 
    retweet_count = int(df["Retweets Received"][index])   
    likes = int(df["Likes Received"][index])
    location = df["Tweet Location"][index]
    coord_lat = df["Lat"][index]
    coord_long= df["Long"][index]
    language = df["Tweet Language"][index]
    #user_id_str = str(df["User Id"][index])
    user_id = int(df["User id int"][index])
    user_id_str = df["User Id"][index].replace('"','')
    user_name = df["Name"][index]
    screen_name = df["Screen Name"][index]
    verified = df["Verified or Non-Verified"][index]
    url = df["Profile URL"][index]
    followers = int(df["User Followers"][index])
    following = int(df["User Following"][index])
    user_creation_date = df["User Account Creation Date"][index]
    sentiment = df["sentiment"][index]
    
    
    try:
        tweety = api.get_status(tweet_id)

        hashtags = tweety.entities['hashtags']

        #print(type(bool(verified)))

        #print(type(user_id_str))

        hashtags = read_hashtags(hashtags)
        #print(index, user_name)

        #print(hashtags)

    except Exception as e:
        if e.api_codes == [88]:
            time.sleep(17 * 60)
            continue
        #if e.api_codes == [63]:
        #    print("error code 63 returned validly")
        
        #print(e)
        #print(type(e))
        #print(dir(e))
        #print(e.api_codes)
        #print(e.api_messages)
        #print(e.response)
        
        
        hashtags = None
        continue
        
    except TweepyException as c:
        print(c)
        print(type(c))
        print(dir(c))
        time.sleep(17 * 60)
        continue
        

    
    #conn = psycopg2.connect(host="localhost",database="TwitterDB",port=5432,user='postgres',password=config.db_password)
    
    #cur = conn.cursor()

    #command = '''INSERT INTO TwitterUser (user_id) VALUES (%s) ON CONFLICT (User_ID) DO NOTHING'''
    
    
    #print(user_id)
    #cur.execute(command,([user_id]))
    
    
    # Commit changes
    #conn.commit()
    
    # Disconnect
    #cur.close()
    #conn.close()
    
    dbConnect(user_id, user_id_str, user_name,screen_name, location, verified,url, followers, following, user_creation_date, tweet_id, tweet_url, created_at, tweet)
    #, created_at, tweet, retweet_count, likes)
    #dbConnect(user_id, user_id_str, user_name, screen_name, location, verified, url, followers, following, user_creation_date,
    #          tweet_id, tweet_url, created_at, tweet, retweet_count, likes, coord_lat,coord_long, language, sentiment)#,
             # hashtags)

    